<a href="https://colab.research.google.com/github/MekdiYilma/HumanitarianCrisisPrediction/blob/main/notebooks/bq_connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd


In [4]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [5]:
%%bigquery --project asana-bq yemen_df
SELECT * 
FROM `gdelt-bq.gdeltv2.events` 
WHERE ActionGeo_CountryCode = 'YM'
LIMIT 20000

In [6]:
yemen_df

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,IsRootEvent,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_FullName,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_ADM2Code,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_FeatureID,Actor2Geo_Type,Actor2Geo_FullName,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_ADM2Code,Actor2Geo_Lat,Actor2Geo_Long,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,927505667,20200605,202006,2020,2020.4247,None,None,None,None,None,None,None,None,None,None,ERI,ERITREA,ERI,None,None,None,None,None,None,None,0,130,130,13,3,-4.4,2,1,2,-5.271318,0,None,None,None,None,NaN,NaN,None,1,Eritrea,ER,ER,None,15.0000,39.0000,ER,4,"Wadah, ?ajjah, Yemen",YM,YM22,32859,16.4224,43.4252,11086319,20200605000000,https://aawsat.com/home/article/2318236/%D8%A7...
1,927453145,20200604,202006,2020,2020.4219,None,None,None,None,None,None,None,None,None,None,YEM,ADEN,YEM,None,None,None,None,None,None,None,1,042,042,04,1,1.9,20,1,20,0.000000,0,None,None,None,None,NaN,NaN,None,1,Saudi Arabia,SA,SA,None,25.0000,45.0000,SA,1,Yemen,YM,YM,None,15.5000,47.5000,YM,20200604183000,https://hournews.net/news-101869.htm
2,927496050,20200604,202006,2020,2020.4219,None,None,None,None,None,None,None,None,None,None,IRN,IRAN,IRN,None,None,None,None,None,None,None,1,051,051,05,1,3.4,4,1,4,-1.095890,0,None,None,None,None,NaN,NaN,None,1,Saudi Arabia,SA,SA,None,25.0000,45.0000,SA,4,"Iyad, Shabwah, Yemen",YM,YM05,32968,14.9833,46.8500,-3188161,20200604230000,https://www.al-madina.com/article/688409/%D9%8...
3,927271488,20200604,202006,2020,2020.4219,None,None,None,None,None,None,None,None,None,None,GBR,UNITED KINGDOM,GBR,None,None,None,None,None,None,None,1,051,051,05,1,3.4,10,1,10,-0.668151,0,None,None,None,None,NaN,NaN,None,1,Yemen,YM,YM,None,15.5000,47.5000,YM,1,Yemen,YM,YM,None,15.5000,47.5000,YM,20200604000000,https://www.okaz.com.sa/news/local/2027016
4,927271595,20200604,202006,2020,2020.4219,None,None,None,None,None,None,None,None,None,None,YEMGOV,YEMEN,YEM,None,None,None,None,GOV,None,None,1,020,020,02,1,3.0,10,1,10,2.397260,0,None,None,None,None,NaN,NaN,None,1,Yemen,YM,YM,None,15.5000,47.5000,YM,1,Yemen,YM,YM,None,15.5000,47.5000,YM,20200604000000,https://www.okaz.com.sa/news/local/2027007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,759102329,20180525,201805,2018,2018.3973,UAF,FIGHTER,None,None,None,None,None,UAF,None,None,YEMUAF,YEMEN,YEM,None,None,None,None,UAF,None,None,0,195,195,19,4,-10.0,4,1,4,-4.705882,4,"Majz, Sa‘dah, Yemen",YM,YM15,32925,17.1333,43.4667,-3778677,4,"Majz, Sa‘dah, Yemen",YM,YM15,32925,17.1333,43.4667,-3778677,4,"Majz, Sa‘dah, Yemen",YM,YM15,32925,17.1333,43.4667,-3778677,20180525170000,http://almoslim.net/news/288359
19996,758976345,20180525,201805,2018,2018.3973,YEMCVL,YEMENI,YEM,None,None,None,None,CVL,None,None,BUS,AIRLINE,None,None,None,None,None,BUS,None,None,1,138,138,13,3,-7.0,10,1,10,-17.582418,4,"Baqim, Sa‘dah, Yemen",YM,YM15,32922,17.3971,43.4774,10647221,4,"Baqim, Sa‘dah, Yemen",YM,YM15,32922,17.3971,43.4774,10647221,4,"Baqim, Sa‘dah, Yemen",YM,YM15,32922,17.3971,43.4774,10647221,20180525080000,http://www.alalam.ir/news/3580726
19997,758906668,20180525,201805,2018,2018.3973,MIL,ARMY,None,None,None,None,None,MIL,None,None,None,None,None,None,None,None,Non

In [4]:
from google.colab import files
yemen_df.to_csv('yemen_batch1.csv') 
files.download('yemen_batch1.csv')

In [5]:
# instead of re-running the query for every load, import the csv saved df locally
from google.colab import files
uploaded = files.upload()

Saving yemen_batch1.csv to yemen_batch1.csv


In [10]:
import io
yemen_df = pd.read_csv(io.BytesIO(uploaded['yemen_batch1.csv']),index_col=[0])

In [11]:
yemen_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 61 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GLOBALEVENTID          20000 non-null  int64  
 1   SQLDATE                20000 non-null  int64  
 2   MonthYear              20000 non-null  int64  
 3   Year                   20000 non-null  int64  
 4   FractionDate           20000 non-null  float64
 5   Actor1Code             18266 non-null  object 
 6   Actor1Name             18266 non-null  object 
 7   Actor1CountryCode      10817 non-null  object 
 8   Actor1KnownGroupCode   830 non-null    object 
 9   Actor1EthnicCode       35 non-null     object 
 10  Actor1Religion1Code    308 non-null    object 
 11  Actor1Religion2Code    104 non-null    object 
 12  Actor1Type1Code        9313 non-null   object 
 13  Actor1Type2Code        699 non-null    object 
 14  Actor1Type3Code        24 non-null     object 
 15  Ac

In [12]:
yemen_df.columns

Index(['GLOBALEVENTID', 'SQLDATE', 'MonthYear', 'Year', 'FractionDate',
       'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode',
       'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code',
       'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code',
       'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
       'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code',
       'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent',
       'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass',
       'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone',
       'Actor1Geo_Type', 'Actor1Geo_FullName', 'Actor1Geo_CountryCode',
       'Actor1Geo_ADM1Code', 'Actor1Geo_ADM2Code', 'Actor1Geo_Lat',
       'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type',
       'Actor2Geo_FullName', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code',
       'Actor2Geo_ADM2Code', 'Actor2Geo_Lat', 'Act

In [16]:
features = ['SQLDATE', 'MonthYear', 'Year', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 
       'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code', 
       'Actor1Type1Code',  'Actor2Code',
       'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
       'Actor2EthnicCode', 'Actor2Religion1Code', 
       'Actor2Type1Code',  'IsRootEvent',
       'EventCode',  'QuadClass',
       'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone',
       'Actor1Geo_Type', 'Actor1Geo_CountryCode',
       'Actor1Geo_ADM1Code',   'Actor2Geo_Type',
       'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code',
        'ActionGeo_Type', 
       'ActionGeo_CountryCode', 'ActionGeo_ADM1Code', 
        'DATEADDED','SOURCEURL']

In [17]:
len(yemen_df.columns), len(features)

(61, 36)

In [18]:
yemen_df.loc[:,['EventCode']]

,EventCode
0,130
1,42
2,51
3,51
4,20
...,...
19995,195
19996,138
19997,190
19998,51


In [54]:
from google.colab import files
uploaded = files.upload()

Saving event_code_lookup.csv to event_code_lookup (1).csv


In [55]:
uploaded

{'event_code_lookup.csv': b'\xef\xbb\xbf01      MAKE PUBLIC STATEMENT\r\n"010     Make statement, not specified below"\r\n011     Decline comment\r\n012     Make pessimistic comment\r\n013     Make optimistic comment\r\n014     Consider policy option\r\n015     Acknowledge or claim responsibility\r\n016     Deny responsibility\r\n017     Engage in symbolic act\r\n018     Make empathetic comment\r\n019     Express accord\r\n02      APPEAL\r\n"020     Appeal, not specified below"\r\n"021     Appeal for material cooperation, not specified below"\r\n0211    Appeal for economic cooperation\r\n0212    Appeal for military cooperation\r\n0213    Appeal for judicial cooperation\r\n0214    Appeal for intelligence\r\n"022     Appeal for diplomatic cooperation, such as policy support"\r\n"023     Appeal for aid, not specified below"\r\n0231    Appeal for economic aid\r\n0232    Appeal for military aid\r\n0233    Appeal for humanitarian aid\r\n0234    Appeal for military protection or peacekeeping\

In [69]:
import io
event_code_lookup = pd.read_csv(io.BytesIO(uploaded['event_code_lookup.csv']),header=None, names=['Event'])

In [70]:
event_code_lookup

,Event
0,01 MAKE PUBLIC STATEMENT
1,"010 Make statement, not specified below"
2,011 Decline comment
3,012 Make pessimistic comment
4,013 Make optimistic comment
...,...
305,202 Engage in mass killings
306,203 Engage in ethnic cleansing
307,"204 Use weapons of mass destruction, not s..."
308,"2041 Use chemical, biological, or radiologi..."


In [67]:
event_code_lookup.iloc[52,:][0].split(' ', 1)[1][4:]

'Express intent to institute political reform, not specified below'

In [85]:
event_code_lookup['EventCode'] = event_code_lookup.Event.str.split(' ', n=1, expand=True)[0]
event_code_lookup['EventName'] = event_code_lookup.Event.str.split(' ', n=1, expand=True)[1].str.slice(start=4)

In [86]:
event_code_lookup.head()

,Event,EventCode,EventName
0,01 MAKE PUBLIC STATEMENT,01,MAKE PUBLIC STATEMENT
1,"010 Make statement, not specified below",010,"Make statement, not specified below"
2,011 Decline comment,011,Decline comment
3,012 Make pessimistic comment,012,Make pessimistic comment
4,013 Make optimistic comment,013,Make optimistic comment


In [107]:
event_code_lookup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Event      310 non-null    object
 1   EventCode  310 non-null    object
 2   EventName  310 non-null    object
dtypes: object(3)
memory usage: 7.4+ KB


In [88]:
yemen = yemen_df[features]
yemen

,SQLDATE,MonthYear,Year,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,IsRootEvent,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_ADM2Code,Actor2Geo_Type,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_ADM2Code,ActionGeo_Type,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,DATEADDED,SOURCEURL
0,20200605,202006,2020,None,None,None,None,None,None,None,None,None,None,ERI,ERITREA,ERI,None,None,None,None,None,None,None,0,130,130,13,3,-4.4,2,1,2,-5.271318,0,None,None,None,1,ER,ER,None,4,YM,YM22,32859,20200605000000,https://aawsat.com/home/article/2318236/%D8%A7...
1,20200604,202006,2020,None,None,None,None,None,None,None,None,None,None,YEM,ADEN,YEM,None,None,None,None,None,None,None,1,042,042,04,1,1.9,20,1,20,0.000000,0,None,None,None,1,SA,SA,None,1,YM,YM,None,20200604183000,https://hournews.net/news-101869.htm
2,20200604,202006,2020,None,None,None,None,None,None,None,None,None,None,IRN,IRAN,IRN,None,None,None,None,None,None,None,1,051,051,05,1,3.4,4,1,4,-1.095890,0,None,None,None,1,SA,SA,None,4,YM,YM05,32968,20200604230000,https://www.al-madina.com/article/688409/%D9%8...
3,20200604,202006,2020,None,None,None,None,None,None,None,None,None,None,GBR,UNITED KINGDOM,GBR,None,None,None,None,None,None,None,1,051,051,05,1,3.4,10,1,10,-0.668151,0,None,None,None,1,YM,YM,None,1,YM,YM,None,20200604000000,https://www.okaz.com.sa/news/local/2027016
4,20200604,202006,2020,None,None,None,None,None,None,None,None,None,None,YEMGOV,YEMEN,YEM,None,None,None,None,GOV,None,None,1,020,020,02,1,3.0,10,1,10,2.397260,0,None,None,None,1,YM,YM,None,1,YM,YM,None,20200604000000,https://www.okaz.com.sa/news/local/2027007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,20200408,202004,2020,SAU,SAUDI ARABIA,SAU,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,190,190,19,4,-10.0,5,1,5,0.310559,1,SA,SA,None,0,None,None,None,1,YM,YM,None,20200408220000,https://english.alarabiya.net/en/coronavirus/2...
19996,20200408,202004,2020,MED,SPOKESMAN,None,None,None,None,None,MED,None,None,None,None,None,None,None,None,None,None,None,None,0,030,030,03,1,4.0,6,1,6,-5.747126,1,SA,SA,None,0,None,None,None,1,YM,YM,None,20200408224500,https://www.upi.com/Top_News/World-News/2020/0...
19997,20200409,202004,2020,YEM,YEMEN,YEM,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,190,190,19,4,-10.0,1,1,1,-2.445652,1,SA,SA,None,0,None,None,None,4,YM,YM16,32956,20200409014500,https://english.alarabiya.net/en/coronavirus/2...
19998,20200409,202004,2020,MED,SPOKESMAN,None,None,None,None,None,MED,None,None,None,None,None,None,None,None,None,None,None,None,1,010,010,01,1,0.0,1,1,1,-8.333333,1,SA,SA,None,0,None,None,None,1,YM,YM,None,20200409023000,https://www.dw.com/en/saudi-led-coalition-decl...


In [92]:
yemen.SQLDATE = pd.to_datetime(yemen.SQLDATE, format='%Y%m%d', errors='ignore')

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [105]:
yemen.DATEADDED = pd.to_datetime(yemen.DATEADDED,  format='%Y%m%d%H%M%S', errors='ignore')

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
